In [26]:
import transformers, tensorflow as tf, keras
print("Transformers:", transformers.__version__)
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)


Transformers: 4.44.2
TensorFlow: 2.20.0
Keras: 3.11.3


In [ ]:
#pip install flask

Note: you may need to restart the kernel to use updated packages.


In [28]:
from flask import Flask, request, jsonify
import threading
from sentence_transformers import SentenceTransformer
import json
import requests
from typing import Union
import os
from contextlib import redirect_stdout, redirect_stderr
import logging
from typing import Dict, List
import together
import torch
import os
import subprocess
import signal
import sys
import logging
import httpx
from openai import OpenAI, DefaultHttpxClient

In [ ]:
#pip install weaviate-client

Note: you may need to restart the kernel to use updated packages.


In [29]:
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.query import Filter
from typing import List
from tqdm import tqdm
import joblib
import weaviate
import re
from weaviate.util import generate_uuid5
from pprint import pprint
import os

In [ ]:
#!pip install -U FlagEmbedding

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mteb 1.12.39 requires numpy<2.0.0,>=1.0.0, but you have numpy 2.3.3 which is incompatible.


In [30]:
from flask import Flask, request, jsonify
import threading
import json
from FlagEmbedding import FlagReranker
import numpy as np
import torch
import threading
import logging
from utils import generate_embedding

In [31]:
reranker = FlagReranker('BAAI/bge-reranker-base', cache_dir = '.models/', use_fp16=True)

In [32]:
import flask_app

In [33]:
from utils import (
    suppress_subprocess_output, 
    generate_with_single_input, 
    print_object_properties
)

In [34]:
with suppress_subprocess_output():
    client = weaviate.connect_to_embedded(
        persistence_data_path="./.collections",
        environment_variables={
            "ENABLE_API_BASED_MODULES": "true", # Enable API based modules 
            "ENABLE_MODULES": 'text2vec-transformers, reranker-transformers', # We will be using a transformer model
            "TRANSFORMERS_INFERENCE_API":"http://127.0.0.1:5000/", # The endpoint the weaviate API will be using to vectorize
            "RERANKER_INFERENCE_API":"http://127.0.0.1:5000/" # The endpoint the weaviate API will be using to rerank
        }
    )

WeaviateStartUpError: Embedded DB did not start because processes are already listening on ports http:8079 and grpc:50050use weaviate.connect_to_local(port=8079, grpc_port=50050) to connect to the existing instance

In [35]:
data = joblib.load("data.joblib")
print_object_properties(data[0])

place: Grand Canyon
state: Arizona
description: A stunning canyon with vast vistas and incredible geology.
best_season_to_visit: Spring, Fall
attractions: South Rim, Havasu Falls, Skywalk
budget: Moderate
user_ratings: 4.8
last_updated: 2023-10-01T00:00:00Z



In [36]:
vectorizer_config = [Configure.NamedVectors.text2vec_transformers(
                name="vector", # This is the name you will need to access the vectors of the objects in your collection
                source_properties=['place', 'state', 'description', 'best_season_to_visit', 'attractions', 'budget'], # which properties should be used to generate a vector, they will be appended to each other when vectorizing
                vectorize_collection_name = False, # This tells the client to not vectorize the collection name. 
                                                   # If True, it will be appended at the beginning of the text to be vectorized
                inference_url="http://127.0.0.1:5000", # Since we are using an API based vectorizer, you need to pass the URL used to make the calls 
                                                       # This was setup in our Flask application
            )]

In [37]:
# Delete the collection in case it exists
if client.collections.exists("example_collection"):
    client.collections.delete("example_collection")

In [38]:
if not client.collections.exists('example_collection'): # Creates only if the collection does not exist
    collection = client.collections.create(
            name='example_collection',
            vectorizer_config=vectorizer_config, # The config we defined before,
            reranker_config=Configure.Reranker.transformers(), # The reranker config

            properties=[  # Define properties
            Property(name="place",vectorize_property_name=True,data_type= DataType.TEXT),
            Property(name="state",vectorize_property_name=True, data_type=DataType.TEXT),
            Property(name="description",vectorize_property_name=True, data_type=DataType.TEXT),
            Property(name="best_season_to_visit",vectorize_property_name=True, data_type=DataType.TEXT),
            Property(name="attractions",vectorize_property_name=True, data_type=DataType.TEXT),
            Property(name="budget",vectorize_property_name=True, data_type=DataType.TEXT),
            Property(name="user_ratings", data_type=DataType.NUMBER),
            Property(name="last_updated", data_type=DataType.DATE),

        ]
        )
else:
    collection = client.collections.get("example_collection")

/opt/conda/lib/python3.11/site-packages/weaviate/warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(


In [39]:
print(collection)

<weaviate.Collection config={
  "name": "Example_collection",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false,
    "auto_tenant_activation": false
  },
  "properties": [
    {
      "name": "place",
      "description": null,
      "data_type": "text",
      "index_filterable": true,
      "index_range_filters": false,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": null,
      "vectorizer": null,
      "vectorizer_configs": {
        "text2vec-transformers": {
          "skip": false,
          "vectorize_property_

In [40]:
try:
    collection = client.collections.create(
        name='example_collection',

        vectorizer_config=vectorizer_config, # The config we defined before,
    
        properties=[  # Define properties
        Property(name="place",vectorize_property_name=True,data_type= DataType.TEXT),
        Property(name="state",vectorize_property_name=True, data_type=DataType.TEXT),
        Property(name="description",vectorize_property_name=True, data_type=DataType.TEXT),
        Property(name="best_season_to_visit",vectorize_property_name=True, data_type=DataType.TEXT),
        Property(name="attractions",vectorize_property_name=True, data_type=DataType.TEXT),
        Property(name="budget",vectorize_property_name=True, data_type=DataType.TEXT),
        Property(name="user_ratings", data_type=DataType.NUMBER),
        Property(name="last_updated", data_type=DataType.DATE),
                 
    ]
    )
except Exception as e:
    print(e)

Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Example_collection already exists'}]}.


In [41]:
client.collections.list_all().keys()

dict_keys(['Example_collection'])

In [42]:
os.environ['TOGETHER_API_KEY'] = "76d81fd6490bf6aa4277b46347bf1af6b951e468bedd5eef3abc37b851d75b31"

In [44]:
from together import Together

# Set up a batch process with specified fixed size and concurrency
with collection.batch.fixed_size(batch_size=1, concurrent_requests=1) as batch:
    # Iterate over a subset of the dataset
    for document in tqdm(data): # tqdm is a library to show progress bars
            # Generate a UUID based on the article_content text for unique identification
            uuid = generate_uuid5(document)

            # Add the object to the batch with properties and UUID. 
            # properties expects a dictionary with the keys being the properties.
            batch.add_object(
                properties=document,
                uuid=uuid,
            )

  0%|          | 0/20 [00:00<?, ?it/s]{'message': 'Failed to send all objects in a batch of 1', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: fail with status 500: name \'Together\' is not defined")'}
{'message': 'Failed to send 1 objects in a batch of 1. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
{'message': 'Failed to send all objects in a batch of 1', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: fail with status 500: name \'Together\' is not defined")'}
{'message': 'Failed to send 1 objects in a batch of 1. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
{'message': 'Failed to send all objects in a batch of 1', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: fail with sta